In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import FinanceDataReader as fdr
import json

In [10]:
symbol_df = pd.read_csv('./nasdaq_screener_1692248735523.csv')
symbol_df = symbol_df.sort_values(by=['Market Cap'], ascending=False)
symbol_df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
21,AAPL,Apple Inc. Common Stock,$176.57,-0.88,-0.496%,2.760536e+12,United States,1980.0,46875999,Technology,Computer Manufacturing
4543,MSFT,Microsoft Corporation Common Stock,$320.40,-0.78,-0.243%,2.380496e+12,United States,1986.0,20669221,Technology,Computer Software: Prepackaged Software
3058,GOOG,Alphabet Inc. Class C Capital Stock,$129.11,-1.16,-0.89%,1.627948e+12,United States,2004.0,17532541,Technology,Computer Software: Programming Data Processing
3059,GOOGL,Alphabet Inc. Class A Common Stock,$128.70,-1.08,-0.832%,1.622778e+12,United States,2004.0,25201147,Technology,Computer Software: Programming Data Processing
419,AMZN,Amazon.com Inc. Common Stock,$135.07,-2.60,-1.889%,1.393619e+12,United States,1997.0,41610987,Consumer Discretionary,Catalog/Specialty Distribution
...,...,...,...,...,...,...,...,...,...,...,...
7370,WRB^G,W.R. Berkley Corporation 4.25% Subordinated De...,$19.00,-0.11,-0.576%,NaN,United States,NaN,11170,NaN,NaN
7371,WRB^H,W.R. Berkley Corporation 4.125% Subordinated D...,$17.11,-0.27,-1.554%,NaN,United States,NaN,6944,NaN,NaN
7384,WSO/B,Watsco Inc.,$367.15,0.00,0.00%,NaN,United States,NaN,2,NaN,NaN
7432,XFLT^A,XAI Octagon Floating Rate & Alternative Income...,$24.92,-0.03,-0.12%,NaN,United States,NaN,6149,NaN,NaN


In [11]:
PERIOD = '1y'

ticker = symbol_df['Symbol'][:100]
data = []
def f(x, q1, q2, q3):
    if x < q1: return 0
    elif x < q2: return 1
    elif x < q3: return 2
    else: return 3
for t in ticker:
    try:
        df = yf.Ticker(t).history(period=PERIOD)
        df = df.drop(['Dividends', 'Stock Splits'], axis=1)
        df.index = df.index.map(lambda x: str(x)[:10])
        df['Change'] = 0
        for i in range(1, len(df)):
            df.loc[df.index[i], 'Change'] = (df.iloc[i]['Close'] - df.iloc[i - 1]['Close']) / df.iloc[i - 1]['Close']
        # 가장 첫 날은 change를 구할 수 없으므로 drop처리
        df = df.drop(df.index[0])
        desc = df.describe()
        Q1 = desc.loc['25%', 'Change']
        Q2 = desc.loc['50%', 'Change']
        Q3 = desc.loc['75%', 'Change']
        df['Class'] = df['Change'].map(lambda x: f(x, Q1, Q2, Q3))

        data.append({
            "stock": t,
            "data": json.loads(df.to_json(orient='index'))
        })
    except:
        print(t)
    
data = json.dumps(data, indent=2)
data = json.loads(data)
with open('./data.json', 'w') as file:
    json.dump(data, file, indent=2)

Failed to get ticker 'BRK/A' reason: Expecting value: line 1 column 1 (char 0)
BRK/A: No price data found, symbol may be delisted (period=1y)


KeyError: "['Dividends', 'Stock Splits'] not found in axis"